In [67]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.layers.core import Dense, Dropout, Activation
from keras import callbacks
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import math
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [95]:
data = pd.read_csv('../../raw/LiChuan/TotalDataSet.csv')
submit = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

In [105]:
data_new=data[['sale_date', 'class_id', 'sale_quantity']]

In [104]:
data['month']

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       1.0
12       1.0
13       1.0
14       1.0
15       1.0
16       1.0
17       1.0
18       1.0
19       1.0
20       1.0
21       1.0
22       1.0
23       1.0
24       1.0
25       1.0
26       1.0
27       1.0
28       1.0
29       1.0
        ... 
9910    11.0
9911    11.0
9912    11.0
9913    11.0
9914    11.0
9915    11.0
9916    11.0
9917    11.0
9918    11.0
9919    11.0
9920    11.0
9921    11.0
9922    11.0
9923    11.0
9924    11.0
9925    11.0
9926    11.0
9927    11.0
9928    11.0
9929    11.0
9930    11.0
9931    11.0
9932    11.0
9933    11.0
9934    11.0
9935    11.0
9936    11.0
9937    11.0
9938    11.0
9939    11.0
Name: month, Length: 9940, dtype: float64

In [102]:
data[data['sale_date']=='2012-01']

,TR_0,TR_1,TR_4,TR_5,TR_5;4,TR_6,TR_7,TR_8,TR_8;7,TR_9,...,sale_quantity_total_last_6M_max,sale_quantity_total_last_6M_min,sale_quantity_total_last_6M_mean,sale_quantity_total_last_6M_std,sale_quantity_total_last_12M_sum,last_12M_sale_class_count_sum,sale_quantity_total_last_12M_max,sale_quantity_total_last_12M_min,sale_quantity_total_last_12M_mean,sale_quantity_total_last_12M_std
0,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,1,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,1,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,1,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,0,0,1,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,0,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# 去掉 2012 年数据
data = data[data['year'] != 2012]
labels = data['sale_quantity']
data = data.drop(['sale_date', 'class_id', 'sale_quantity'], axis=1)

In [90]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = data[:-560]
test_1 = data[-560:-420]
labels_1 = labels[:-560]
test_labels_1 = labels[-560:-420]

# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = data[:-420]
test_2 = data[-420:-280]
labels_2 = labels[:-420]
test_labels_2 = labels[-420:-280]

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = data[:-280]
test_3 = data[-280:-140]
labels_3 = labels[:-280]
test_labels_3 = labels[-280:-140]

In [91]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = train_1.as_matrix()
test_1 = test_1.as_matrix()
labels_1 = labels_1.as_matrix()
test_labels_1 = test_labels_1.as_matrix()
# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = train_2.as_matrix()
test_2 = test_2.as_matrix()
labels_2 = labels_2.as_matrix()
test_labels_2 = test_labels_2.as_matrix()

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = train_3.as_matrix()
test_3 = test_3.as_matrix()
labels_3 = labels_3.as_matrix()
test_labels_3 = test_labels_3.as_matrix()

In [92]:
scaler = MinMaxScaler(feature_range=(0, 1))

train_1 = scaler.fit_transform(train_1)
test_1 = scaler.fit_transform(test_1)

train_2 = scaler.fit_transform(train_2)
test_2 = scaler.fit_transform(test_2)

train_3 = scaler.fit_transform(train_3)
test_3 = scaler.fit_transform(test_3)

In [93]:
# labels_1 = scaler.fit_transform(labels_1)
# test_labels_1 = scaler.fit_transform(test_labels_1)

# labels_2 = scaler.fit_transform(labels_2)
# test_labels_2 = scaler.fit_transform(test_labels_2)

# labels_3 = scaler.fit_transform(labels_3)
# test_labels_3 = scaler.fit_transform(test_labels_3)

In [94]:
train_1

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.90887911],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.90887911],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.90887911]])

In [73]:

train_1 = train_1.reshape((train_1.shape[0], 1, train_1.shape[1]))
test_1 = test_1.reshape((test_1.shape[0], 1, test_1.shape[1]))
train_2 = train_2.reshape((train_2.shape[0], 1, train_2.shape[1]))
test_2 = test_2.reshape((test_2.shape[0], 1, test_2.shape[1]))
train_3 = train_3.reshape((train_3.shape[0], 1, train_3.shape[1]))
test_3 = test_3.reshape((test_3.shape[0], 1, test_3.shape[1]))

In [21]:
# BATCH_START = 0
# TIME_STEPS = 20 #取20步
# BATCH_SIZE = 50
# INPUT_SIZE = 1 #输入的值
# OUTPUT_SIZE = 1 #输出预测的值
# CELL_SIZE = 20
# LR = 0.006

(7700, 1, 204)

In [75]:
model = Sequential()
# model.add(LSTM(
#     batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
#     output_dim=CELL_SIZE,
#     return_sequences=True,      # True: output at all steps. False: output as last step.
#     stateful=True,              # True: the final state of batch1 is feed into the initial state of batch2
# ))
model.add(LSTM(64, input_shape=(train_1.shape[1],train_1.shape[2])))
model.add(Dropout(.1))
model.add(Dense(64))
model.add(Dropout(.2))
model.add(Dense(16))
model.add(Dropout(.2))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer='adam', metrics=['mse'])

In [76]:
model.fit(train_1, labels_1, nb_epoch=30, batch_size=10, verbose=2)

Epoch 1/30
 - 4s - loss: 118172.1572 - mean_squared_error: 118172.1572
Epoch 2/30
 - 2s - loss: 49488.5203 - mean_squared_error: 49488.5203
Epoch 3/30
 - 2s - loss: 47199.6614 - mean_squared_error: 47199.6614
Epoch 4/30
 - 2s - loss: 46941.2241 - mean_squared_error: 46941.2241
Epoch 5/30
 - 2s - loss: 42309.3385 - mean_squared_error: 42309.3385
Epoch 6/30
 - 2s - loss: 44751.3412 - mean_squared_error: 44751.3412
Epoch 7/30
 - 2s - loss: 42706.2462 - mean_squared_error: 42706.2462
Epoch 8/30
 - 2s - loss: 40331.3087 - mean_squared_error: 40331.3087
Epoch 9/30
 - 2s - loss: 40896.0232 - mean_squared_error: 40896.0232
Epoch 10/30
 - 2s - loss: 44675.8705 - mean_squared_error: 44675.8705
Epoch 11/30
 - 2s - loss: 36705.5976 - mean_squared_error: 36705.5976
Epoch 12/30
 - 2s - loss: 36574.4394 - mean_squared_error: 36574.4394
Epoch 13/30
 - 2s - loss: 41262.8304 - mean_squared_error: 41262.8304
Epoch 14/30
 - 2s - loss: 38689.8529 - mean_squared_error: 38689.8529
Epoch 15/30
 - 2s - loss: 3

In [77]:
model.save('my_model.h5')   # HDF5 file, you have to pip3 install h5py if don't have it

In [78]:
pred_test_labels_1=model.predict(test_1)

In [79]:
math.sqrt(((pred_test_labels_1 - test_labels_1)**2).mean())

987.9534512003171